In [ ]:
# Illustration of the Fiber Section
import openseespy.opensees as ops
import opsvis as opsv
import matplotlib.pyplot as plt
import numpy as np

# Units: N mm t MPa

ops.wipe()
ops.model('basic', '-ndm', 2, '-ndf', 3)

# Material Properties
# ======================================================
fc = 26.38  # Concerte Compressive Strength
fyl = 497   # Yielding Strength of Longitudinal Bars
fyt = 459.5 # Yielding Strength of Transverse Ties
# ======================================================

# Specimen Geometry (Square Section)
# ======================================================
Height = 1400 # Height of the Column
Width = 400   # Width of the Column
Cover = 34    # Thickness of Concrete Cover
# ======================================================

# Reinforcement Configuration
# ======================================================
dlongi = 19.05 # Diameter of the Longitudinal Bars
dtrans = 6.35  # Diamter of the Transverse Bars
strans = 54    # Spacing of the Transverse Bars
# ======================================================

# Axial Force (Approximately 0.1 * fc * Ag)
# ======================================================
axial = -450e3 # Magnitude of the axial force
# ======================================================

# Material Definition
# ======================================================
## Unconfined Concrete
## ops.uniaxialMaterial('Concrete01', matTag, fpc, epsc0, fpcu, epsU)
ops.uniaxialMaterial('Concrete01', 1, fc, 0.002, 0.2 * fc, 0.006)

## Confined Concrete
## ops.uniaxialMaterial('Concrete01', matTag, fpc, epsc0, fpcu, epsU)
ops.uniaxialMaterial('Concrete01', 2, 1.2 * fc, 0.005, 0.8 * 1.2 * fc, 0.015)

## Longitudinal Bars
## ops.uniaxialMaterial('Steel02', matTag, Fy, E0, b, *params, a1=a2*Fy/E0, a2=1.0, a3=a4*Fy/E0, a4=1.0, sigInit=0.0)
ops.uniaxialMaterial('Steel02', 3, fyl, 200e3, 0.01, 20, 0.925, 0.15)
# ======================================================

fib_sec_2 = [['section', 'Fiber', 1],
             ['patch', 'rect', 2, 10, 10, Width / 2 - Cover, Width / 2 - Cover, -Width / 2 + Cover, -Width / 2 + Cover],
             ['patch', 'rect', 1, 10, 1, Width / 2, Width / 2, -Width / 2, Width / 2 - Cover],
             ['patch', 'rect', 1, 10, 1, Width / 2, -Width / 2 + Cover, -Width / 2, -Width / 2],
             ['patch', 'rect', 1, 1, 8, -Width / 2 + Cover, Width / 2 - Cover, -Width / 2, -Width / 2 + Cover],
             ['patch', 'rect', 1, 1, 8, Width / 2, Width / 2 - Cover, Width / 2 - Cover, -Width / 2 + Cover],
             ['layer', 'straight', 3, 4, 0.25 * np.pi * dlongi**2, -Width / 2 + Cover + dlongi / 2, Width / 2 - Cover - dlongi / 2, Width / 2 - Cover - dlongi / 2, Width / 2 - Cover - dlongi / 2],
             ['layer', 'straight', 3, 4, 0.25 * np.pi * dlongi**2, -Width / 2 + Cover + dlongi / 2, -Width / 2 + Cover + dlongi / 2, Width / 2 - Cover - dlongi / 2, -Width / 2 + Cover + dlongi / 2],
             ['layer', 'straight', 3, 2, 0.25 * np.pi * dlongi**2, -Width / 2 + Cover + dlongi / 2, Width / 2 - Cover - 1 / 3 * (Width - 2 * Cover - dlongi) - dlongi / 2, Width / 2 - Cover - dlongi / 2, Width / 2 - Cover - 1 / 3 * (Width - 2 * Cover - dlongi) - dlongi / 2],
             ['layer', 'straight', 3, 2, 0.25 * np.pi * dlongi**2, -Width / 2 + Cover + dlongi / 2, -Width / 2 + Cover + 1 / 3 * (Width - 2 * Cover - dlongi) + dlongi / 2, Width / 2 - Cover - dlongi / 2, -Width / 2 + Cover + 1 / 3 * (Width - 2 * Cover - dlongi) + dlongi / 2]]
matcolor = ['r', 'lightgrey', 'gold', 'w', 'w', 'w']
opsv.plot_fiber_section(fib_sec_2, matcolor=matcolor)
plt.axis('equal')

In [ ]:
# Comparison of the Numerical and Experimental Results
import numpy as np
import matplotlib.pyplot as plt

# 'Mo and Wang, 2000 Test C3-1'
expdata = np.loadtxt('exp.txt', skiprows = 2)
Fexp = np.append(np.zeros(1),expdata[:, 1])
dexp = np.append(np.zeros(1),expdata[:, 0])

numdata = np.loadtxt('out_01SITC_full.txt')
Fnum = np.append(np.zeros(1),numdata[:, 0]) / 1e3 # Convertion from N to kN
dnum = np.append(np.zeros(1),numdata[:, 1])

plt.plot(dexp[0:500], Fexp[0:500])
plt.plot(dnum[0:1400] / 0.03937, Fnum[0:1400] / 0.2248 * 1000, alpha = 0.8)

plt.xlabel('Displacement(mm)')
plt.ylabel('Lateral Force(kN)')
plt.title('Concrete01SITC')


This command is used to construct a uniaxial material object for capturing strain penetration effects at the column-to-footing, column-to-bridge bent caps, and wall-to-footing intersections. In these cases, the bond slip associated with strain penetration typically occurs along a portion of the anchorage length. This model can also be applied to the beam end regions, where the strain penetration may include slippage of the bar along the entire anchorage length, but the model parameters should be chosen appropriately.

This model is for fully anchored steel reinforcement bars that experience bond slip along a portion of the anchorage length due to strain penetration effects, which are usually the case for column and wall longitudinal bars anchored into footings or bridge joints

```opensees
uniaxialMaterial Bond_SP01 $matTag $Fy $Sy $Fu $Su $b $R

$matTag	integer tag identifying material
$Fy	Yield strength of the reinforcement steel
$Sy	Rebar slip at member interface under yield stress. (see NOTES below)
$Fu	Ultimate strength of the reinforcement steel
$Su	Rebar slip at the loaded end at the bar fracture strength (30~40)*Sy
$b	Initial hardening ratio in the monotonic slip vs. bar stress response (0.3~0.5)
$R	Pinching factor for the cyclic slip vs. bar response (0.5~1.0)
```

***NOTE: Need parameters in [ksi kip inch]!!!***

in which, the rebar slip $S_y$ can be obtained by

$$
S_y = 2.54\bigg( \displaystyle{\frac{d_b}{8437}}\displaystyle{\frac{F_y}{\sqrt{f_c^{'}}}}(2\alpha + 1) \bigg)^{1/\alpha} + 0.34 \tag{1}
$$

In Equation (1), $d_b$ is the diameter of the rebar; $\alpha$ is the parameter used in the local bond-slip relation and can be taken as 0.4 in accordance with CEB-FIP Model Code 90. Note the length is measured in mm, while the strength in MPa.

In [ ]:
# Rebar Diameter (mm) and Material Strength (MPa)
# ======================================================
db = 19.05
Fy = 497
fc = 26.38
# ======================================================

alpha = 0.4

# Rebar Slip (mm)
# ======================================================
Sy = 2.54 * (db / 8437 * Fy / (fc**0.5) * (2 * alpha + 1))**(1 / alpha) + 0.34
# ======================================================

Sy

The zero-length section element in OpenSees is assumed to have ***a unit length*** such that the element deformations (i.e., elongation and rotation) are equal to the section deformations (i.e., axial strain and curvature). The material model for the steel fibers in the zero-length section element represents the bar slip instead of strain for a given bar stress. The uniaxial material model Bond_SP01 is developed for steel fibers in the zero-length section elements.

Similar to the model proposed for the steel fibers, a material model describing the monotonic response and hysteretic rules is also required for the concrete fibers. The combination of using the zero-length section element and enforcing the plane section assumption at the end of a flexural member impose **high deformations to the extreme concrete fibers in the zero-length element**. These deformations would likely correspond to concrete compressive strains significantly greater than the strain capacity stipulated by typical confined concrete models. Such high compressive strains at the end of flexural members are possible because of additional confinement effects expected from the adjoining members and because of complex localized deformation at the member end. Without further proof, it is suggested that the concrete fibers in the zero-length section element follow a concrete model in OpenSees. **To accommodate the large deformations expected to the extreme concrete fibers in the zero-length element, this concrete model may be assumed to follow a perfectly plastic behavior once the concrete strength reduces to 80% of the confined compressive strength**. A parametric study has indicated that the simulation results would not be very sensitive to the compressive strain chosen to trigger the perfectly plastic behavior for the concrete fibers in the zero-length section element.